In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from prophet import Prophet

from src.features import build_features
from src.models import train_model

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [3]:
DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J955",
    "M348",
    "T820",
    "T821",
    "Z450",
]

In [16]:
egresos_torax = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_torax["FECHA_EGRESO"] = pd.to_datetime(
    egresos_torax["ANO_EGRESO"].astype(str)
    + "-"
    + egresos_torax["MES_EGRESO"].astype(str)
    + "-"
    + egresos_torax["DIA_EGRESO"].astype(str),
    format="%Y-%m-%d",
)

# 1. Proyección de Egresos Hospitalarios para Diagnósticos más frecuentes en el INT

En este cuadernillo se quiere crear un modelo para estimar la proyección de egresos
hospitalarios para los diagnósticos más relevantes para el INT hasta el año 2035. Para realizar
la proyección se utilizará el modelo Prophet, y se hará una proyección por día.

Para este análisis se obviará el año 2019 para entrenar/testear el modelo. Esto, ya que es un
año anómalo.

In [33]:
df = egresos_torax.query("ANO_EGRESO <= 2019")
df = df[df["DIAG1"].isin(DIAGS_RELEVANTES)]
df = df.drop(
    columns={
        "ANO_EGRESO",
        "MES_EGRESO",
        "DIA_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "GLOSA_ESTABLECIMIENTO_SALUD",
    }
)


# Agrupa por día y genera un calendario completo hasta el 2019 (imputa días faltantes)
df = df.groupby("FECHA_EGRESO").sum().resample("D").sum()
df = df.drop(columns="DIAG1")

## 1.1 Preprocesamiento para Modelo Prophet

El modelo prophet requiere 2 columnas para hacer la proyección

In [42]:
df_prophet = (
    df.copy()
    .reset_index()[["FECHA_EGRESO", "n_egresos"]]
    .rename(columns={"FECHA_EGRESO": "ds", "n_egresos": "y"})
)